In this short programming assignment we will look into applications of word embeddings in similarity search and nearest neighbors. We will also look at creating a video based on tSNE embeddings of images.

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import csv

from sklearn.manifold import TSNE
import cv2

## -1. Create a video from images
Download any 1000 or more 'appropriate' and publicly available images from the web. This could be part of a data set or something specific that you picked up or are interested in.

We discussed using tSNE to find image embeddings for these images. Apply the tSNE library to these images and construct low-dimensional embeddings for the images. Use these embeddings to then:

a) Start at any random image in the data set 

b) Sequentially chain the next image to the previous image using a scoring function/probability based on the tSNE embedding. So you want to chain the most similar image to the current one and so on. Choose a frame rate that is appropriate to convert this chain of images into a video. Your video shouldn't be more than 3 minutes long.

c) Upload this video to youtube and share a link with your submission. 

d) Feel free to share your video to Discord to see what cool videos we come up with!

In [9]:
flow_path = "/Users/qingchuanhou/Desktop/Samples/flower_train"
n = 1000

def Img_list(path, width=224, height=224):
    X = list()

    for label in os.listdir(path):
        if not label.startswith('.'):
            label_path = os.path.join(path, label)

            i = 0
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                image = cv2.imread(image_path)
                image_data = Img2data(image, width, height)
                X += [image_data]
                i += 1
                if i > n/5 - 1:
                    break
    return np.array(X)      
    


def Img2data(image, width=224, height=224):
    image_resize = cv2.resize(image, (width, height))
    image_data = np.reshape(image_resize, width*height*3)
    return image_data

def Data2img(image_data, width=224, height=224):
    image = np.reshape(image_data, (width,height,3))
    return image

flowr_list = Img_list(flow_path)

In [10]:
# TSNE in 1d
def tSNE_sort1d(img_list):
    tsne_data = TSNE(n_components=1, init='random').fit_transform(img_list)
    image_sort = [x for _,x in sorted(zip(tsne_data, img_list))]
    return image_sort

In [11]:
def video_out(images, width=224, height=224):
    out = cv2.VideoWriter('tsne_1d.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, (width, height))

    for i in range(len(images)):
        out.write(Data2img(images[i], width, height))

    out.release()

def tSNE_1dvideo(img_list, width=224, height=224):
    image_sort = tSNE_sort1d(img_list)
    video_out(image_sort, width, height)


In [12]:
# tSNE_1dvideo(flowr_list)

In [13]:
# tSNE in 2d
def tSNE_2dvideo(img_list, width=224, height=224):
    tsne_data2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(img_list)
    out = cv2.VideoWriter('tsne_2d.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, (width, height))

    a = int(np.random.randint(len(tsne_data2d)))
    x = tsne_data2d[a][0]
    y = tsne_data2d[a][1]
    tsne_data2d = np.delete(tsne_data2d, a, 0)
    out.write(Data2img(img_list[a], width, height))
    img_list = np.delete(img_list, a, 0)
    for a in range(len(img_list)):
        min_dis = ((x - tsne_data2d[0][0])**2 + (y - tsne_data2d[0][1])**2)**0.5
        m = 0
        for i in range(len(tsne_data2d)):
            if min(min_dis, ((x - tsne_data2d[i][0])**2 + (y - tsne_data2d[i][1])**2)**0.5) < min_dis:
                min_dis = ((x - tsne_data2d[i][0])**2 + (y - tsne_data2d[i][1])**2)**0.5
                m = i
        x = tsne_data2d[m][0]
        y = tsne_data2d[m][1]
        tsne_data2d = np.delete(tsne_data2d, m, 0)
        out.write(Data2img(img_list[m], width, height))
        img_list = np.delete(img_list, m, 0)
    out.release()


In [14]:
tSNE_2dvideo(flowr_list)

In [15]:
# divide video frame and use tSNE sort
video_path = '/Users/qingchuanhou/Desktop/Samples/170724_15_Setangibeach.mp4'

def video_read(path, width=224, height=224):
    cap = cv2.VideoCapture(path)
    X = list()
    while cap.isOpened():
        try:
            ret,frame = cap.read()
            if ret == True:
                image_resize = cv2.resize(frame, (width, height), fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                image_data = np.reshape(image_resize, width*height*3)
                X += [image_data]
            else:
                break
        except Exception as e:
            print(str(e))
    return np.array(X)

In [16]:
video_img_list = video_read(video_path, width=480, height=480)
# video_img_list = np.random.shuffle(video_img_list)
tSNE_2dvideo(video_img_list, width=480, height=480)

Sort flower images using 1d tSNE: https://youtu.be/qEiuoSa4KfU

Sort flower images using 2d tSNE: https://youtu.be/EAUB0SLwrzQ

## Diving into Cheat Sheet of Pandas Data Frame
There are some useful functions for solving problems below when it comes to index and slice data frames. Let's go over them.
More materials can be found here: https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf
1. DataFrame() Construct a dataframe. Use it for putting a numpy ndarray into a dataframe.
1. DataFrame.loc() Purely label-location based indexer for selection by label. Use it for selecting word vectors in the dataframe.     
1. DataFrame.dot() Matrix multiplication with DataFrame. Use it for dot product of word vectors.
1. DataFrame.sort_values() Sort by the values along either axis. Use it for sorting distance short to long.

Below are examples of using these functions. You don't have to code anything in this block, just focus on understanding the functions and how it works in pandas.

In [17]:
# Define a ndarray
d = np.array([[0.1,0.3,0.4,0.5],[0.3,0.4,0.9,0.5],[0.2,0.8,0.7,0.5]], dtype=float, order='F')
print("Define sample word vectors")
print(d)
#Construct a dataframe from ndarray and index each row as word vectors
df = pd.DataFrame(d,index = ['word1','word2','word3'])
print("\nPandas Data Frame for word vectors")
print(df)
#Select word vector1 by its label
print("\nFind the row corresponding to word1")
print(df.loc['word1'])
#Calculate dot product of word vector1 and word vector2
print("\nCalculate the dot product between word1 and word2")
dot_product = df.loc['word2'].dot(df.loc['word1'])
print(dot_product)
#Calculate dot product of word vector1 to the rest of words
print("\nCalculate the dot product between word1 and rest of the words")
words_rest = ['word2','word3']
dot_product2 = df.loc[words_rest].dot(df.loc['word1'])
print(dot_product2)
#Sort Values of dot_product2 by high to low
print("\nSorted dot product values")
print(dot_product2.sort_values(ascending = False))

Define sample word vectors
[[0.1 0.3 0.4 0.5]
 [0.3 0.4 0.9 0.5]
 [0.2 0.8 0.7 0.5]]

Pandas Data Frame for word vectors
         0    1    2    3
word1  0.1  0.3  0.4  0.5
word2  0.3  0.4  0.9  0.5
word3  0.2  0.8  0.7  0.5

Find the row corresponding to word1
0    0.1
1    0.3
2    0.4
3    0.5
Name: word1, dtype: float64

Calculate the dot product between word1 and word2
0.76

Calculate the dot product between word1 and rest of the words
word2    0.76
word3    0.79
dtype: float64

Sorted dot product values
word3    0.79
word2    0.76
dtype: float64


## Dataset Details - Standford's GloVe pre-trained word vectors

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus. The GloVe pre-trained word vectors dataset contains English word vectors pre-trained on the combined Wikipedia 2014 + Gigaword 5th Edition corpora (6B tokens, 400K vocab). All tokens are in lowercase. This dataset contains 50-dimensional, 100-dimensional and 200-dimensional pre trained word vectors. In this problem we are going to use the 50-dimensional dataset. 

## \# 0. Get an overview on what Glove is
Read up the documentation on glove embeddings, esp. where it gets applied here: https://nlp.stanford.edu/projects/glove/

## Load Dataset
Let's load the dataset first. Each row is indexed as a word vector. Dimension of word vectors is 50. How many words are there in this dataset? Print a few words and see what they are. You don't need to code anything here, just understand the data structure.

In [18]:
# Load GloVe pre-trained vectors 
local_file1="/Users/qingchuanhou/Desktop/glove/glove.6B.50d.txt" # Make sure this file exists!
data = csv.reader
df = pd.read_csv(local_file1,sep=' ',index_col=0,header=None,engine='python',error_bad_lines=False, quoting=csv.QUOTE_NONE)
print("dataset shape - Rows: %d, Cols: %d" % (df.shape[0], df.shape[1]))
words = list(df.index)
print("print a few words in the dataset:", words[30:40])

/Users/qingchuanhou/opt/anaconda3/envs/main/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/qingchuanhou/Desktop/glove/glove.6B.50d.txt'

## \# 1. Print the first few 11 rows of the pandas data frame below

In [ ]:
# Your code HERE - It should execute as expected! 
# (Search for a pandas functionality that can help you do this!)
df.head(11)

## \# 2. Words Similarity

Similar words have similar embeddings (or vector values). We can use cosine similarity i.e. cos(u,v) = u.v/(|u||v|) to measure vector similarity. u.v is dot product of vectors, |u| is L2 norm of u. Remember, we spoke about computing similarity based on cosine-similarity (as another alternative to correlation) in class?

1. Normalize matrix df by norm of word vectors. 
1. Define a function to find words similarity to a given word.
1. Use the function defined to find the word in examples that is most similar to "happy".

In [ ]:
## YOUR CODE HERE
# 1a. Calculate norm of word vectors
# What would be the dimension of the vector_norm array?
vector_norm = np.sqrt(np.square(df).sum(axis=1))

# 1b. Normalize matrix df by norm using .div()
dfn = df.div(vector_norm, axis=0)

# 2. Define a function to find words similar to a given word in a normalized dataframe
def find_word_similarity(word, examples, dataframe):
    # Input: word - one string
    #        examples - List of strings
    #        dataframe - An indexed normalized dataframe
    ## YOUR CODE HERE
    # Calculate dot product of each word in examples to the given word, sorted by value high to low
    # Once you have the sorted values of dot products (notice because of normalization, the dot product is the cosine similarity!),
    # obtain the words corresponding to the sorted values and call it similar_words
    df = dataframe

    dot_product = df.loc[examples].dot(df.loc[word])

    similar_words = dot_product.sort_values(ascending = False)

    # Return words similar to the given word
    return similar_words
    
examples = ["sad", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]

# 3.
# Use above function to calculate examples' similarity to happy (both "happy" and words in examples are in dfn)
print(find_word_similarity("happy", examples,dfn))

In sklean library,there is a cosine_similarity fuction that directly calcualtes vectors similarity (you don't need to normalize vectors first). Let's use this function to calculate similarity again to confirm we get same results. 
For more information, please see here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity2 = pd.DataFrame(cosine_similarity(np.array(df.loc['happy']).reshape(1,50),np.array(df.loc[examples])),columns = examples)
print(similarity2.T.sort_values(by=0, ascending = False))

## \# 3. Goodness of similarity
Comment on the how good the glove embeddings are on finding similar words to a given word using cosine similarity? Glove and word2vec embeddings are based on co-occurence of words in senetences across hundreds of thousands of documents on the web. Would this help explain your observations on word similarity?
What if you replace happy with sad, how do the results change?

In [ ]:
examples2 = ["happy", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]
print(find_word_similarity("sad", examples2,dfn))

The result for using Glove embedding to find the similar words is pretty good. There are some words that don't seem to be very related, such as 'hardly' is similar for both 'happy' and 'sad'. I think this may also have something to do with the frequency of words, because all the data are normalized, so although some words are not very frequent, they appear in sentences containing other words, which will cause these words to use cosine have high similarity.Also, in the Glove, what we mean by similar is not just semantically similar, but also the similarty in the kind of words, like 'happy' and 'sad'

## \# 4. Correlations
(This question is more of a reflection and building your intuition on how correlations we spoke in class connects to a real-world data set -  Open ended!)
What are some of the most correlated words from the similarity search you did earlier to the word "happy" and "sad". Likewise, what are some of the most uncorrelated words to "happy" and "sad". Does this make sense? How would you improve the results ? If "happy" were a random variable and "sad" was a random variable - What factors make the correlation between "happy" and "sad" (as you computed above) high?

In the Glove, what we mean by similar is not just semantically similar, but also the similarty in the kind of words. Like 'happy' and 'sad'. It make sense because these words can be exchanged in some sentences. The other words in this sentence will influce this word's vector in Glove. like 'I feel sad' and 'I feel happy' because the influence of the words I and feel makes 'sad' and 'happy' have similar vectors in Glove.

## \# 5. Find nearest neighbourhood

It is helpful to compute the nearest neighbors to a word based on the cosine similarity that we defined earlier, so that given a word we can compute which are the other words which are most similar to it. Sometimes, the nearest neighbors according to this metric reveal overlap of concepts or topics that a word shares. E.g. government might be related to the word politics because they both share topics related to public policy, politicians, parties, elections, etc. The idea is whatever embeddings we are using - word2vec or glove is "hopefully" able to capture these correlations right!

1. Define a function to find the top n similar words to a given word. You can use either dot product of vectors or cosine_simialrity function. Note the search space for words is coming from your pandas data frame (so unlike the similarity problem we worked on earlier, we are not restricted to only a few words to search from - the search space here is the entire vocab captured in your data frame).
1. Find 20 nearest neighborhood for words 'duck' and 'animal'.
1. Find neighborhood intersection of 'duck' and 'animal', to find which words are similar to both 'duck' and 'animal'. This is also related to a similarity measure called "Jaccard Similarity" - Read up on this here: https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
lis = list(dfn.index)
lis.remove('the')
print(lis[0])

In [ ]:
# define a function to find the top n similar words to a given word in the 'df'

# PART 1
def find_most_similar(df, word, n):
    # INPUT: 
    # df: Given Data frame
    # word: String
    # n: Number of similar words to return
    
    # OUTPUT:
    # the list of similar words to return
    
    ## YOUR CODE HERE
    # define and compute the most similar words
    # Use a similarity measure like cosine similarity (like earlier) to do so

    words = list(df.index)
    words.remove(word)

    dot_product = df.loc[words].dot(df.loc[word])

    similar_words = dot_product.sort_values(ascending = False).head(n)
    
    return similar_words


# PART 2
# find top 20 similar words to duck
simil1 = find_most_similar(dfn, "duck", 20)
print('Simillar words to \'duck\' (most simillar first):', list(simil1.index))
print()
# find top 20 similar words to animal
simil2 = find_most_similar(dfn, "animal", 20)
print('Simillar words to \'animal\' (most simillar first):', list(simil2.index))

# PART 3
# find the intersection of simil1 and simil2
# intersection =  (concat function of pandas is helpful here)
# print intersection
intersection = pd.concat([simil1, simil2], axis=1, join='inner')
print(intersection)


word_labels = ['duck', 'animal'] + list(intersection.index)
print(word_labels)


## \# 6 Word analogies

Suppose you know the word vectors for King, Man and Woman. What is your intuitive answer for the 'riddle', King - Man + Woman = ? 
Let's go through below steps to derive the answer for this 'riddle' using the word embeddings.

1. Use vector arithmetic to define a new vector which equals to k - m + w (e.g. king, man and woman combination).
2. Calculate similarity of all the words in the corpus to the new vector and sort them by their similarity high to low. 
3. Return the top n vectors which have the highest similarity to the new vector.
1. Find the answers for the riddles, 
    1. good:bad::up:?
    1. germany:merkel::america:?

In [ ]:
# define a function to solve the problem of x is to y as a is to ?
# 'n' is the number of top words similar to the vector to return
# 'dataframe' is the indexed dataframe that contains all the words

# PART 1,2,3 above (Fill in the missing pieces)
def solve_riddle(x, y, a, n, dataframe):
    ## YOUR CODE HERE
    # calculate the vector of a + y - x, where a, x, y are in dataframe
    df = dataframe
    cal_vec = df.loc[a] + df.loc[y] - df.loc[x]
    
    # calculate distance of words in dataframe to cal_vec, sorted by similarity high to low
    dot_product = df.loc[list(df.index)].dot(cal_vec)

    similarity = dot_product.sort_values(ascending = False).head(n)

    # return top n words and distance that closest to cal_vec
    return similarity 

# Call the solve_riddle function to compute the top answers
print(solve_riddle("man", "woman", "king", 5,df))


In [ ]:
## YOUR CODE HERE
# Solve the other two riddles
# good:bad::up:?
# PART 4
#print solve_riddle()
print(solve_riddle("good", "bad", "up", 5,df))


In [ ]:
# germany:merkel::america:?
#print solve_riddle()
print(solve_riddle("germany", "america", "merkel", 5,df))